In [2]:
import pandas as pd
import re

### Import

In [46]:
# Importing the signature key in two parts

df1 = pd.read_csv('data/bhr1.csv', error_bad_lines=False, sep=';')
columns = df1.columns
df2 = pd.read_csv('data/bhr2.csv', names = columns, error_bad_lines=False, sep=';')
df = pd.concat([df1, df2], ignore_index=True)


C:\Users\hanna\AppData\Local\Temp/ipykernel_1684/440638329.py:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df1 = pd.read_csv('data/bhr1.csv', error_bad_lines=False, sep=';')
C:\Users\hanna\AppData\Local\Temp/ipykernel_1684/440638329.py:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df2 = pd.read_csv('data/bhr2.csv', names = columns, error_bad_lines=False, sep=';')


## Cleaning

In [47]:
df.drop('vw', axis=1, inplace=True)

#### Text 

In [48]:
# getting rid of the ... in the texts
df["text"] = df["text"].str.replace(r'\.{3,}', '')

# getting rid of new lines (oopsies)
df['sys'] = df['sys'].str.replace('\n', '')

C:\Users\hanna\AppData\Local\Temp/ipykernel_1684/248016969.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df["text"] = df["text"].str.replace(r'\.{3,}', '')


#### Backreferencing

In [50]:
# replace idem (=...)

backreference = re.compile(r"idem\s*\(\s*=\s*(.*)\s*\)")

df["backreference"] = df["text"].str.extract(backreference, expand=False)

# Replaces the backreference with the first captured group
df["text"] = df["text"].str.replace(backreference, r"\1")



In [51]:
# find the remaining idems

idem = re.compile(r'\bidem\s*[,.]?\s*')

idem_df = df[df["text"].str.contains(idem)]

In [52]:
# replace the idem without backreference

for i, row in idem_df.iterrows():
    # loop over rows in the df DataFrame above the current row
    for j in range(i-1, -1, -1):

        if not idem.search(df.loc[j, "text"]):

            reference = df.iloc[j].text.split(',')[0]
            
            #replace idem with the actual backreference
            substring = df.iloc[i].text

            modified_substring = idem.sub(f'{reference} ', substring)

            df.loc[i, "text"] = modified_substring
            break



#### Level 1, 2, 3 Texts

In [67]:
def level_text (lev, text, df): 
    df[text]= ''

    indices = df[df.lev == lev].index
    for i in range(len(indices) -1): 
        start = indices[i]
        end = indices[i+1] 

        # add level to all rows below in the tree
        df.loc[start + 1 :end - 1, text][df.lev >= lev] = df.iloc[start].text

    #handle the last case seperately 
    if len(indices) > 0: 
        start = indices[-1]
        end = len(df)
        df.loc[start + 1 :end - 1, text][df.lev >= lev]  = df.iloc[start].text


for i in range(1, 5): 
    level_text(i, 'text_' + str(i), df)





#### Ranges

In [69]:
# Appending all the ranges as rows, to make lookup operations easier later

df.numbis = df.numbis.fillna('')
range_rows = df[df.numbis.str.isdigit()]

for i, row in range_rows.iterrows():

    start = int(re.findall(r'\d+', row.sys)[0])
    end = int(row.numbis)

    for j in range(start, end): 
        new_row = row.copy()
        new_row['sys'] = new_row['sys'].replace(str(start), str(j+1))

        df = df.append(new_row, ignore_index=True)


C:\Users\hanna\AppData\Local\Temp/ipykernel_1684/1229544586.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


# Export 

In [70]:
df.to_csv('data/csv/signatures.csv', index=False)

In [10]:
df = pd.read_csv('data/csv/signatures.csv')

In [11]:
df

,lev,sys,numbis,text,backreference,text_1,text_2,text_3,text_4
0,NaN,NaN,NaN,Bibliotheca Hertziana,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,Systematischer Standortkatalog,NaN,NaN,NaN,NaN,NaN
2,1.0,A,NaN,Handbücherei,NaN,NaN,NaN,NaN,NaN
3,2.0,Aa,NaN,Allgemeine Nachschlagewerke,NaN,Handbücherei,NaN,NaN,NaN
4,3.0,Aa xxx,yyy,Lexika,NaN,Handbücherei,Allgemeine Nachschlagewerke,NaN,NaN
...,...,...,...,...,...,...,...,...,...
55496,5.0,Zt 8846,8850,westeuropäische Länder,NaN,Nachbarwissenschaften,Theologie und Frömmigkeit (Weltreligionen),Christentum,"Frömmigkeit (Predigt, Reliquienkult, Wallfahrt..."
55497,5.0,Zt 8847,8850,westeuropäische Länder,NaN,Nachbarwissenschaften,Theologie und Frömmigkeit (Weltreligionen),Christentum,"Frömmigkeit (Predigt, Reliquienkult, Wallfahrt..."
55498,5.0,Zt 8848,8850,westeuropäische Länder,NaN,Nachbarwissenschaften,Theologie und Frömmigkeit (Weltreligionen),Christentum,"Frömmigkeit (Predigt, Reliquienkult, Wallfahrt..."
55499,5.0,Zt 8849,8850,westeuropäische Länder,NaN,Nachbarwissenschaften,Theologie und Frömmigkeit (Weltreligionen),Christentum,"Frömmigkeit (Predigt, Reliquienkult, Wallfahrt..."


# SyCa database 

In [3]:
syca = pd.read_csv('data/csv/signatures_C_E.csv', sep='\t')

In [4]:
syca.sign.isna().sum()

0

In [13]:
# Artists

def parse_artists(row): 
    field = row.sign
    name = row['name']
    if field.startswith('C'): 
        if len(field) > 1 and field[1] == 'a':
            return 'Italienische Künstler', 'Alte Künstler (geboren vor 1870)', name
        if len(field) > 1 and field[1] == 'm':
            return 'Italienische Künstler', 'Moderne Künstler (geboren nach 1870)', name
        else:
            return None, None, None
    elif field.startswith('W'): 
        if len(field) > 1 and field[1] == 'a':
            return 'Ausseritalienische Künstler', 'Alte Künstler (geboren vor 1870)', name
        if len(field) > 1 and field[1] == 'm':
            return 'Ausseritalienische Künstler', 'Moderne Künstler (geboren nach 1870)', name
        else:
            return None, None, None
    else: 
        return None, None, None
    



In [36]:
syca

,sign,grp,nr,ref,name,comment,author,date,x,text_1,text_2
0,Ca-ABB,Ca,70,NaN,"Abbate, Niccolo dell'",NaN,nk,1987.09.22,NaN,Italienische Künstler,Alte Künstler (geboren vor 1870)
1,Ca-ABB,Ca,105,NaN,"Abbati, Clemente degli",NaN,ni,1984.11.28,NaN,Italienische Künstler,Alte Künstler (geboren vor 1870)
2,Ca-ABB,Ca,110,NaN,"Abbati, Giuseppe",NaN,ni,1984.11.28,NaN,Italienische Künstler,Alte Künstler (geboren vor 1870)
3,Ca-ABB,Ca,150,NaN,"Abbatini, Guido Ubaldo",NaN,gw,1990.03.21,NaN,Italienische Künstler,Alte Künstler (geboren vor 1870)
4,Ca-ABB,Ca,180,NaN,"Abbiati, Filippo",NaN,ni,1984.11.28,NaN,Italienische Künstler,Alte Künstler (geboren vor 1870)
...,...,...,...,...,...,...,...,...,...,...,...
38705,Ca-PRE,NaN,100,NaN,"Premazzi, Luigi",NaN,ms,2023.04.13,NaN,Italienische Künstler,Alte Künstler (geboren vor 1870)
38706,Cm-ALP,NaN,224,NaN,"Alpago Novello, Adriano",NaN,ms,2023.04.14,NaN,Italienische Künstler,Moderne Künstler (geboren nach 1870)
38707,Cm-BIA,NaN,790,NaN,"Bianchetti, Angelo",NaN,ms,2023.04.14,NaN,Italienische Künstler,Moderne Künstler (geboren nach 1870)
38708,Wm-PER,NaN,6900,NaN,"Pers, Isabella",NaN,ms,2023.04.17,NaN,Ausseritalienische Künstler,Moderne Künstler (geboren nach 1870)


In [28]:
artists = pd.DataFrame(columns=df.columns)
rows = []

for i, row in syca.iterrows():
    text_1, text_2, text_3 = parse_artists(row)
    sys = row.sign + ' ' + str(row.nr)
    new_row = {'lev': 3, 'sys': sys, 'text': text_3, 'text_1': text_1, 'text_2': text_2}
    rows.append(new_row)

artists = pd.concat([pd.DataFrame(rows)], ignore_index=True)


In [30]:
df = pd.concat([df, artists], ignore_index=True)

In [31]:
df

,lev,sys,numbis,text,backreference,text_1,text_2,text_3,text_4
0,NaN,NaN,NaN,Bibliotheca Hertziana,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,Systematischer Standortkatalog,NaN,NaN,NaN,NaN,NaN
2,1.0,A,NaN,Handbücherei,NaN,NaN,NaN,NaN,NaN
3,2.0,Aa,NaN,Allgemeine Nachschlagewerke,NaN,Handbücherei,NaN,NaN,NaN
4,3.0,Aa xxx,yyy,Lexika,NaN,Handbücherei,Allgemeine Nachschlagewerke,NaN,NaN
...,...,...,...,...,...,...,...,...,...
94206,3.0,Ca-PRE 100,NaN,"Premazzi, Luigi",NaN,Italienische Künstler,Alte Künstler (geboren vor 1870),NaN,NaN
94207,3.0,Cm-ALP 224,NaN,"Alpago Novello, Adriano",NaN,Italienische Künstler,Moderne Künstler (geboren nach 1870),NaN,NaN
94208,3.0,Cm-BIA 790,NaN,"Bianchetti, Angelo",NaN,Italienische Künstler,Moderne Künstler (geboren nach 1870),NaN,NaN
94209,3.0,Wm-PER 6900,NaN,"Pers, Isabella",NaN,Ausseritalienische Künstler,Moderne Künstler (geboren nach 1870),NaN,NaN


In [ ]:
# Mapping a and b artists to the start number of their signature
a_artists = {'Bernini': 1920, 'Giotto': 660, 'Leonardo Da Vinci': 220, 'Michelangelo': 20, 'Raffael': 140, 'Tiepolo, Giov. Batt.': 10, 'Tiziano': 10}
b_artists = {'Angelico (Fra Angelico)': 310, 'Bellini, Giovanni': 770, 'Borromini': 530, 'Boticelli': 180, 'Bramante': 270, 'Canaletto, Bernardo': 110,
             'Canova': 980, 'Caravaggio': 316, 'Cellini': 290, 'Correggio': 1080, 'Donatello': 70, 'Duccio di Buoninsegna': 90,
             'Franceschi, Piero': 250, 'Ghiberti, Lorenzo': 40, 'Ghirlandaio': 350, 'Giorgione': 580, 'Guardi, Francesco': 320, 
             'Mantegna': 980, 'Masaccio': 20, 'Palladio': 320, 'Perugino': 1200 ,'Reni, Guido': 70, 'Tintoretto, Jacopo': 220, 'Veronese': 670}

rows = []

for a, start in a_artists.items():
    sys = 'Ca-' + a[:3].capitalize + ' '

    row_0 = {'lev': 4, 'sys': sys + str(start), 'text': 'Bibliographien', 'text_3': a, 'text_2': 'Alte Künstler (geboren vor 1870)', 'text_1': 'Italienische Künstler'}
    row_1 = {'lev': 4, 'sys': sys + str(start + 1), 'text': 'Quellenpublikationen', 'text_3': a, 'text_2': 'Alte Künstler (geboren vor 1870)', 'text_1': 'Italienische Künstler'}
    row_2 = {'lev': 4, 'sys': sys + str(start + 2), 'text': 'Sammelschriften', 'text_3': a, 'text_2': 'Alte Künstler (geboren vor 1870)', 'text_1': 'Italienische Künstler'}
    row_3 = {'lev': 4, 'sys': sys + str(start + 3), 'text': 'Ausstellungskataloge', 'text_3': a, 'text_2': 'Alte Künstler (geboren vor 1870)', 'text_1': 'Italienische Künstler'}
    row_4 = {'lev': 4, 'sys': sys + str(start + 4), 'text': 'Vollbiographien und Oeuvreverzeichnisse des Gesamtlebenswerkes', 'text_3': a, 'text_2': 'Alte Künstler (geboren vor 1870)', 'text_1': 'Italienische Künstler'}
    row_5 = {'lev': 4, 'sys': sys + str(start + 5), 'text': 'Teilbiographien und Oeuvreverzeichnisse einzelner Arbeitsperioden', 'text_3': a, 'text_2': 'Alte Künstler (geboren vor 1870)', 'text_1': 'Italienische Künstler'}
    row_6 = {'lev': 4, 'sys': sys + str(start + 6), 'text': 'Teilbiographien und Oeuvreverzeichnisse einzelner Arbeitsgebiete', 'text_3': a, 'text_2': 'Alte Künstler (geboren vor 1870)', 'text_1': 'Italienische Künstler'}
    row_7 = {'lev': 4, 'sys': sys + str(start + 7), 'text': 'Werkmonographien', 'text_3': a, 'text_2': 'Alte Künstler (geboren vor 1870)', 'text_1': 'Italienische Künstler'}
    row_8 = {'lev': 4, 'sys': sys + str(start + 8), 'text': 'Einzelfragen', 'text_3': a, 'text_2': 'Alte Künstler (geboren vor 1870)', 'text_1': 'Italienische Künstler'}

    

